<a href="https://colab.research.google.com/github/cricci3/Salaries_clustering/blob/main/Project2_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Data Analytics
Assignment 2 - Group 2
Salaries (clustering)

Students: Claudio Ricci (claudio.ricci@usi.ch) and Pernille Bergesen (pernille.bergesen@usi.ch)
'''

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

#loading the data from the san-francisco datasets
dataset_file_2011 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2011.csv'
dataset_file_2012 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2012.csv'
dataset_file_2013 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2013.csv'
dataset_file_2014 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2014.csv'

#just for testing:
#printing first two lines of 2011 dataset
print("san-francisco-2011.csv: ")
print(pd.read_csv(dataset_file_2011).head(2))

#printing first two lines of 2012 dataset
print("san-francisco-2012.csv: ")
print(pd.read_csv(dataset_file_2012).head(2))

#printing first two lines of 2013 dataset
print("san-francisco-2013.csv: ")
print(pd.read_csv(dataset_file_2013).head(2))

#printing first two lines of 2014 dataset
print("san-francisco-2014.csv: ")
print(pd.read_csv(dataset_file_2014).head(2))

san-francisco-2011.csv: 
    Employee Name                                       Job Title   Base Pay  \
0  NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY  167411.18   
1    GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)  155966.02   

   Overtime Pay  Other Pay      Benefits  Total Pay  Total Pay & Benefits  \
0          0.00  400184.25  Not Provided  567595.43             567595.43   
1     245131.88  137811.38  Not Provided  538909.28             538909.28   

   Year  Notes         Agency  
0  2011    NaN  San Francisco  
1  2011    NaN  San Francisco  
san-francisco-2012.csv: 
    Employee Name                     Job Title   Base Pay  Overtime Pay  \
0  Gary Altenberg  Lieutenant, Fire Suppression  128808.87     220909.48   
1    Gregory Suhr               Chief of Police  302578.00          0.00   

   Other Pay  Benefits  Total Pay  Total Pay & Benefits  Year  Notes  \
0   13126.31  44430.12  362844.66             407274.78  2012    NaN   
1   1897